## 🧠 The Core Idea

👉 **One agent = one decision domain**
👉 **Agents talk via files + state, not chat**

## 👥 Agent Team (Minimal but Powerful)

### 1️⃣ **Planner Agent**

**Responsibility:** *What needs to be done*

* Reads:

  * project structure
  * Java version
  * Spring version
* Outputs:

  * migration plan (`plan.json`)
  * Java target (17)
  * Spring target

📄 Output example:

```json
{
  "java_target": 17,
  "spring_boot_target": "3.2.x",
  "modules": ["web", "security"],
  "risk_level": "medium"
}
```

---

### 2️⃣ **Build Migration Agent**

**Responsibility:** *Make project buildable on Java 17*

* Modifies:

  * `build.gradle`
  * `gradle-wrapper.properties`
* Upgrades:

  * Gradle
  * Spring Boot
  * dependency versions

🧠 Deterministic → almost zero hallucination

---

### 3️⃣ **Source Classifier Agent**

**Responsibility:** *Decide WHAT needs rewriting*

For **each Java file**, it decides:

* `NO_CHANGE`
* `MINOR_FIX`
* `MAJOR_REWRITE`
* `REMOVE`

📄 Output:

```json
{
  "UserController.java": "MAJOR_REWRITE",
  "HealthCheck.java": "NO_CHANGE",
  "SecurityConfig.java": "MAJOR_REWRITE"
}
```

---

### 4️⃣ **Rewrite Agent**

**Responsibility:** *Rewrite only what’s necessary*

Rules:

* Same package
* Same class name
* Same public APIs
* Java 17 + Spring 3 compatible
* javax → jakarta
* No “creative refactoring”

This agent is **strictly controlled**.

---

### 5️⃣ **Validation / Repair Agent**

**Responsibility:** *Make it compile*

* Runs:

  ```bash
  ./gradlew build
  ```
* Feeds errors back to:

  * Rewrite Agent
* Stops only when:

  * Build passes OR
  * Error is non-fixable

---

## 🔄 How they work together (Flow)

```
Planner
   ↓
Build Agent
   ↓
Source Classifier
   ↓
Rewrite Agent
   ↓
Validation Agent
   ↺ (loops back if errors)
```

No human intervention needed until the end.

---





In [11]:
from agents.planner_agent import run_planner_agent
print("▶ Starting planner agent script")
PROJECT_ROOT = "/Users/raghuveerrohine/Java/Java-Migration-Demo"
OUTPUT_PLAN = "migration_state/migration_plan.json"
print("▶ Calling planner agent")
run_planner_agent(PROJECT_ROOT, OUTPUT_PLAN)


▶ Starting planner agent script
▶ Calling planner agent
🚀 Planner agent started
📂 Project root resolved: /Users/raghuveerrohine/Java/Java-Migration-Demo
📄 Reading build.gradle...
   ✔ build.gradle loaded (684 chars)
📄 Reading settings.gradle...
   ✔ settings.gradle loaded (41 chars)
🌲 Generating project tree...
   ✔ Project tree generated (1261 chars)
🧠 Building planner prompt...
   ✔ Prompt built (2857 chars)
📡 Calling LLM (this may take time)...
   ✔ LLM response received in 4.41s
   ✔ LLM response: {
  "current_java": 8,
  "target_java": 17,
  "current_spring_boot": "2.7.18",
  "target_spring_boot": "3.2.x",
  "project_type": "spring-boot",
  "modules": ["main"],
  "uses_web": true,
  "uses_security": false,
  "uses_jpa": true,
  "risk_level": "medium"
}
🧾 Parsing LLM response as JSON...
   ✔ JSON parsed successfully
💾 Preparing output path: migration_state
💾 Writing migration plan to file: migration_state/migration_plan.json
✅ Migration plan generated successfully
⏱ Total time: 4.4

In [12]:
from agents.build_migration_agent import run_build_migration_agent

PROJECT_ROOT = "/Users/raghuveerrohine/Java/Java-Migration-Result/Java-Migration-Demo"
PLAN_PATH = "migration_state/migration_plan.json"

run_build_migration_agent(PROJECT_ROOT, PLAN_PATH)

🔧 Running Build Migration Agent
→ Target Java: 17
→ Target Spring Boot: 3.2.x
✅ Gradle wrapper upgraded to 8.6
✅ build.gradle regenerated via LLM
✅ Build migration completed


In [1]:
from agents.source_classifier import classify_sources

SRC_PATH = "/Users/raghuveerrohine/Java/Java-Migration-Demo/src"
OUTPUT = "migration_state/source_classification.json"

classify_sources(SRC_PATH, OUTPUT)

[CLASSIFIED] /Users/raghuveerrohine/Java/Java-Migration-Demo/src/test/java/com/exp/JavaMigrationDemoApplicationTests.java -> NO_CHANGE
[CLASSIFIED] /Users/raghuveerrohine/Java/Java-Migration-Demo/src/main/java/com/exp/JavaMigrationDemoApplication.java -> MAJOR_REWRITE
[CLASSIFIED] /Users/raghuveerrohine/Java/Java-Migration-Demo/src/main/java/com/exp/dto/UserRequest.java -> MAJOR_REWRITE
[CLASSIFIED] /Users/raghuveerrohine/Java/Java-Migration-Demo/src/main/java/com/exp/filters/LoggingFilter.java -> MAJOR_REWRITE
[CLASSIFIED] /Users/raghuveerrohine/Java/Java-Migration-Demo/src/main/java/com/exp/repo/User.java -> MAJOR_REWRITE


In [2]:
from agents.rewrite_agent import run_rewrite_agent

OLD_PROJECT = "/Users/raghuveerrohine/Java/Java-Migration-Demo"
NEW_PROJECT = "/Users/raghuveerrohine/Java/Java-Migration-Result/Java-Migration-Demo"
CLASSIFICATION = "migration_state/source_classification.json"

run_rewrite_agent(
    old_project_root=OLD_PROJECT,
    new_project_root=NEW_PROJECT,
    classification_path=CLASSIFICATION
)

✍️ Rewritten: JavaMigrationDemoApplication.java
✍️ Rewritten: UserRequest.java
✍️ Rewritten: LoggingFilter.java
✍️ Rewritten: User.java
✅ Rewrite Agent completed


In [3]:
from agents.validation_agent import run_validation_agent

OLD_PROJECT = "/Users/raghuveerrohine/Java/Java-Migration-Demo"
NEW_PROJECT = "/Users/raghuveerrohine/Java/Java-Migration-Result/Java-Migration-Demo"

run_validation_agent(
    old_project_root=OLD_PROJECT,
    new_project_root=NEW_PROJECT
)


🔍 Starting Validation Agent

🔁 Build attempt 1
✅ BUILD SUCCESSFUL


In [ ]:
print("API key found and looks good so far! guardrails")